In [ ]:
import molsysmt as msm
from molsysmt import pyunitwizard as puw
import omembrane as omem
import openmm as mm
from openmm import unit as u
import numpy as np
import math
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
molsys = msm.convert('memb_popc_100_stage_1.msmpk')

In [ ]:
msm.info(molsys)

In [ ]:
#msm.view(molsys, with_water_as='licorice')

In [ ]:
msm.molecular_mechanics.get_potential_energy()